In [22]:
import pandas as pd
import numpy as np
import random

# Data

In [5]:
df = pd.read_excel('GA_task.xlsx', skiprows=1, header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,R,T,R,T,R,T,R,T,R,T,...,R,T,R,T,R,T,R,T,R,T
1,9,22,5,17,5,24,8,40,9,14,...,7,21,9,24,6,37,9,26,10,36
2,3,49,10,13,4,28,5,47,7,45,...,3,10,10,40,7,22,1,39,2,46
3,1,47,7,23,1,18,5,47,6,30,...,8,22,9,24,3,39,6,31,4,16
4,9,30,4,29,5,32,7,18,2,20,...,7,27,7,22,2,49,10,13,5,18
5,5,21,6,27,4,17,7,27,10,42,...,7,50,4,26,4,19,1,20,2,21
6,3,19,8,33,10,31,2,43,1,29,...,10,29,8,17,3,38,10,21,2,33
7,2,18,2,49,8,50,7,33,4,21,...,10,33,7,12,6,24,5,25,8,46
8,7,45,9,39,7,34,1,10,9,35,...,10,11,6,42,8,20,4,18,3,40
9,6,10,1,15,5,38,1,35,7,43,...,5,13,2,24,4,48,6,40,7,27


In [10]:
df.isna().sum().sum()

0

In [12]:
data = df.iloc[1: ]
data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1,9,22,5,17,5,24,8,40,9,14,...,7,21,9,24,6,37,9,26,10,36
2,3,49,10,13,4,28,5,47,7,45,...,3,10,10,40,7,22,1,39,2,46
3,1,47,7,23,1,18,5,47,6,30,...,8,22,9,24,3,39,6,31,4,16
4,9,30,4,29,5,32,7,18,2,20,...,7,27,7,22,2,49,10,13,5,18
5,5,21,6,27,4,17,7,27,10,42,...,7,50,4,26,4,19,1,20,2,21
6,3,19,8,33,10,31,2,43,1,29,...,10,29,8,17,3,38,10,21,2,33
7,2,18,2,49,8,50,7,33,4,21,...,10,33,7,12,6,24,5,25,8,46
8,7,45,9,39,7,34,1,10,9,35,...,10,11,6,42,8,20,4,18,3,40
9,6,10,1,15,5,38,1,35,7,43,...,5,13,2,24,4,48,6,40,7,27
10,8,39,9,12,1,29,7,15,1,12,...,6,28,2,39,6,20,3,40,7,43


In [13]:
n_rows, n_cols = data.shape
n_jobs = n_cols // 2
n_rows, n_cols, n_jobs

(11, 100, 50)

In [ ]:
jobs = []
for job_id in range(n_jobs):
    r_col = job_id * 2
    t_col = r_col + 1
    job_operations = []
    for operation_id in range(n_rows):
        R = data.iloc[operation_id, r_col]
        T = data.iloc[operation_id, t_col]
        job_operations.append({'resource': int(R), 'time': int(T)})
    jobs.append(job_operations)

In [18]:
len(jobs)

50

In [21]:
jobs[0]

[{'resource': 9, 'time': 22},
 {'resource': 3, 'time': 49},
 {'resource': 1, 'time': 47},
 {'resource': 9, 'time': 30},
 {'resource': 5, 'time': 21},
 {'resource': 3, 'time': 19},
 {'resource': 2, 'time': 18},
 {'resource': 7, 'time': 45},
 {'resource': 6, 'time': 10},
 {'resource': 8, 'time': 39},
 {'resource': 5, 'time': 46}]

In [54]:
POP_SIZE = 30           # Number of different solutions in the generation
GENERATIONS = 100       # Number of epochs
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.2
TOURNAMENT_SIZE = 3     # Number of best solutions to select from
NUM_SWAPS = 5

In [55]:
num_jobs = len(jobs)
num_ops_per_job = len(jobs[0])
total_ops = num_jobs * num_ops_per_job

In [56]:
def random_chromosome(num_jobs, num_ops_per_job):
    chrom = []
    for job_id in range(num_jobs):
        chrom.extend([job_id] * num_ops_per_job)
    random.shuffle(chrom)
    return chrom

def decode_and_evaluate(chromosome, jobs):
    num_jobs = len(jobs)
    num_ops_per_job = len(jobs[0])
    job_next_op = [0] * num_jobs # tracks the next operation to schedule for each job
    resource_available = {} # dictionary that holds the time when each resource becomes available
    job_end_time = [0] * num_jobs

    # each resource starts available at time 0
    for job_id in range(num_jobs):
        for job in jobs[job_id]:
            resource_available[job['resource']] = 0

    for job_id in chromosome:
        op_idx = job_next_op[job_id]
        if op_idx >= num_ops_per_job:
            continue  # skip if all operations for this job are done
        op = jobs[job_id][op_idx]
        res = op['resource']
        time_for_op = op['time']
        
        # The operation can start when both the job and the resource are available
        start = max(job_end_time[job_id], resource_available[res]) # holds the earliest possible start time
        finish = start + time_for_op
        
        job_end_time[job_id] = finish
        resource_available[res] = finish
        job_next_op[job_id] += 1 # move to the next operation for this job

    return max(job_end_time)

def tournament(pop, fitnesses, k):
    selected = random.sample(list(zip(pop, fitnesses)), k) # select k random solutions with their fitness
    selected.sort(key=lambda x: x[1]) # sort by fitness (lowest time)
    return selected[0][0] # best chromosome

def crossover(parent1, parent2):
    size = len(parent1)
    a, b = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[a:b] = parent1[a:b]
    fill = [x for x in parent2 if x not in child[a:b] or child[a:b].count(x) < parent2.count(x)]
    idx = 0
    for i in range(size):
        if child[i] is None:
            child[i] = fill[idx]
            idx += 1
    return child

def mutate(chrom, num_swaps):
     # swap two random genes
     for _ in range(num_swaps):
        a, b = random.sample(range(len(chrom)), 2)
        chrom[a], chrom[b] = chrom[b], chrom[a]


In [57]:
population = [random_chromosome(num_jobs, num_ops_per_job) for _ in range(POP_SIZE)]

for gen in range(GENERATIONS):
    fitnesses = [decode_and_evaluate(chrom, jobs) for chrom in population]
    new_population = []
    for _ in range(POP_SIZE):
        
        parent1 = tournament(population, fitnesses, TOURNAMENT_SIZE)
        parent2 = tournament(population, fitnesses, TOURNAMENT_SIZE)
        while parent2 == parent1:
            parent2 = tournament(population, fitnesses, TOURNAMENT_SIZE)
            
        if random.random() < CROSSOVER_RATE:
            child = crossover(parent1, parent2)
        else:
            child = parent1[:]
            
        if random.random() < MUTATION_RATE:
            mutate(child, NUM_SWAPS)
            
        new_population.append(child)
        
    population = new_population

In [58]:
fitnesses = [decode_and_evaluate(chrom, jobs) for chrom in population]
best_idx = np.argmin(fitnesses)
best_chrom = population[best_idx]
best_time = fitnesses[best_idx]
print("Best time:", best_time)
print("Best chromosome:", best_chrom)

Best time: 1339
Best chromosome: [45, 2, 45, 16, 19, 33, 48, 46, 8, 19, 46, 2, 24, 48, 24, 44, 28, 34, 48, 44, 16, 13, 49, 32, 45, 19, 19, 39, 5, 24, 6, 0, 12, 16, 49, 24, 28, 12, 6, 29, 10, 49, 24, 28, 20, 20, 12, 26, 6, 29, 10, 42, 45, 26, 9, 19, 19, 25, 48, 6, 25, 13, 48, 46, 0, 7, 33, 29, 5, 34, 44, 5, 8, 45, 10, 14, 29, 39, 13, 18, 16, 39, 13, 19, 48, 13, 2, 32, 2, 5, 48, 13, 16, 39, 1, 13, 19, 9, 33, 15, 4, 16, 39, 13, 19, 9, 48, 13, 2, 32, 2, 5, 48, 13, 46, 25, 18, 48, 33, 15, 4, 4, 0, 12, 14, 39, 34, 29, 24, 39, 49, 29, 28, 19, 14, 9, 15, 6, 20, 33, 24, 45, 10, 42, 4, 46, 18, 16, 13, 9, 9, 48, 13, 5, 34, 44, 48, 46, 0, 45, 10, 14, 7, 39, 18, 16, 29, 10, 2, 32, 2, 10, 2, 32, 2, 2, 39, 48, 13, 46, 2, 5, 48, 13, 46, 5, 0, 45, 15, 2, 5, 48, 13, 48, 25, 48, 33, 15, 4, 16, 13, 9, 48, 13, 2, 32, 2, 48, 13, 46, 25, 18, 5, 48, 13, 46, 5, 34, 18, 5, 8, 33, 10, 14, 29, 39, 24, 29, 16, 19, 13, 19, 9, 15, 4, 6, 13, 9, 48, 13, 2, 32, 2, 5, 48, 15, 13, 25, 18, 48, 33, 15, 0, 0, 0, 12, 5, 48, 

In [59]:
pop_sizes = [20, 40, 60]
generations_list = [100, 200, 300]
mutation_rates = [0.1, 0.3]
num_swaps_list = [2, 5]

results = []

for POP_SIZE in pop_sizes:
    for GENERATIONS in generations_list:
            for MUTATION_RATE in mutation_rates:
                for NUM_SWAPS in num_swaps_list:

                    population = [random_chromosome(num_jobs, num_ops_per_job) for _ in range(POP_SIZE)]
                    best_time = float('inf')
                    best_chrom = None

                    for gen in range(GENERATIONS):
                        fitnesses = [decode_and_evaluate(chrom, jobs) for chrom in population]
                        new_population = []
                        for _ in range(POP_SIZE):
                            parent1 = tournament(population, fitnesses, TOURNAMENT_SIZE)
                            parent2 = tournament(population, fitnesses, TOURNAMENT_SIZE)
                            while parent2 == parent1:
                                parent2 = tournament(population, fitnesses, TOURNAMENT_SIZE)
                            if random.random() < CROSSOVER_RATE:
                                child = crossover(parent1, parent2)
                            else:
                                child = parent1[:]
                            if random.random() < MUTATION_RATE:
                                mutate(child, NUM_SWAPS)
                            new_population.append(child)
                        population = new_population

                    fitnesses = [decode_and_evaluate(chrom, jobs) for chrom in population]
                    idx = np.argmin(fitnesses)
                    best_time = fitnesses[idx]
                    best_chrom = population[idx]

                    results.append({
                        'POP_SIZE': POP_SIZE,
                        'GENERATIONS': GENERATIONS,
                        'CROSSOVER_RATE': CROSSOVER_RATE,
                        'MUTATION_RATE': MUTATION_RATE,
                        'NUM_SWAPS': NUM_SWAPS,
                        'BEST_TIME': best_time,
                        'BEST_CHROM': best_chrom
                    })

df_results = pd.DataFrame(results)

In [62]:
df_results.sort_values(by='BEST_TIME', ascending=True)

,POP_SIZE,GENERATIONS,CROSSOVER_RATE,MUTATION_RATE,NUM_SWAPS,BEST_TIME,BEST_CHROM
23,40,300,0.9,0.3,5,840,"[15, 45, 37, 18, 48, 48, 48, 45, 17, 18, 22, 1..."
28,60,200,0.9,0.1,2,885,"[31, 46, 9, 16, 27, 8, 23, 10, 30, 23, 29, 28,..."
26,60,100,0.9,0.3,2,953,"[29, 21, 17, 27, 43, 4, 9, 37, 27, 3, 33, 33, ..."
34,60,300,0.9,0.3,2,1009,"[25, 21, 26, 13, 10, 43, 22, 48, 25, 47, 43, 2..."
11,20,300,0.9,0.3,5,1015,"[26, 33, 35, 5, 39, 33, 33, 33, 23, 23, 45, 7,..."
35,60,300,0.9,0.3,5,1022,"[13, 40, 26, 45, 13, 25, 13, 4, 15, 25, 43, 25..."
21,40,300,0.9,0.1,5,1087,"[12, 20, 47, 32, 32, 20, 36, 21, 47, 6, 39, 32..."
29,60,200,0.9,0.1,5,1114,"[38, 28, 13, 29, 23, 11, 23, 7, 27, 4, 4, 29, ..."
17,40,200,0.9,0.1,5,1120,"[23, 4, 23, 19, 4, 22, 46, 49, 46, 49, 34, 22,..."
30,60,200,0.9,0.3,2,1130,"[15, 43, 1, 7, 35, 19, 15, 11, 25, 3, 35, 40, ..."
